# Assignment 3
* Please use an example dataset of travel diary data (linked trip) which are uploaded on LearnUS 
    * Class Files -> Travel diary -> Travel_data_Linked.csv
* For detail information of each column, check “Travel_data_Linked_description.xlsx”)

# Set up

## Import

In [172]:
import os
import pandas as pd
import numpy as np
import seaborn as sns

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn import tree

from pydotplus import graph_from_dot_data
from sklearn.tree import export_graphviz

## Load Dataset

In [173]:
data_dir = 'D:/OneDrive - 연세대학교 (Yonsei University)/Lectures/2022-2_모빌리티데이터분석및활용/dataset'

In [174]:
data_file = 'Travel_data_Linked.csv'

In [175]:
data_path = os.path.join(data_dir, data_file)

In [176]:
df = pd.read_csv(data_path)

In [177]:
df.head(3)

,No,HH_ID,HHsize,HHsize5,HHinc,Ncars,P_ID,HHrel,Age,Gender,...,Trip_ID,Trip_purpG,Trip_d_P,Trip_d_hh,Trip_d_mm,Trip_d_D,Trip_a_P,Trip_a_hh,Trip_a_mm,Trip_a_D
0,1,4889,2,2,4,1,1,1,36,1,...,1.0,1.0,1.0,6.0,30.0,1.138060e+09,1.0,7.0,30.0,1.168064e+09
1,2,4889,2,2,4,1,1,1,36,1,...,2.0,5.0,2.0,9.0,0.0,1.168064e+09,2.0,9.0,40.0,1.138060e+09
2,3,4889,2,2,4,1,2,2,36,2,...,1.0,1.0,1.0,8.0,0.0,1.138060e+09,1.0,8.0,30.0,1.156055e+09


# Q1
* Develop a decision tree model of forecasting trip purpose by considering the following conditions:
    - Use the variables included in the dataset only (Do not use external data)
    - Create new variables (i.e. features) using the variables if necessary (e.g., travel time, etc.)
    - Apply F1-score as the measurement of the performance
*  Submit Jupyter Notebook file of your model and results
    - (You don’t need to submit the tree structure, but F1-score should be stated in the results.)

## Create Derived Variable

### Create Dummy Variables

In [178]:
def Dummy_car_ownership(Ncars):

    if Ncars == 1: # Car own
        return 1
    elif Ncars == 2: # Do not own
        return 0
    else:
        pass

In [179]:
def Dummy_HHrel_1(HHrel):
    if HHrel == 1: # 'Householder'
        return 1
    else:
        return 0
    
def Dummy_HHrel_2(HHrel):
    if HHrel == 2: # 'Spouse'
        return 1
    else:
        return 0
    
def Dummy_HHrel_3(HHrel):
    if HHrel == 3: # 'Children'
        return 1
    else:
        return 0
    
def Dummy_HHrel_4(HHrel):
    if HHrel == 4: #'Parents'
        return 1
    else:
        return 0
    
def Dummy_HHrel_5(HHrel):
    if HHrel == 5: #'etc.'
        return 1
    else:
        return 0

In [180]:
def Dummy_Gender(Gender):
    if Gender == 1: # Male
        return 1
    elif Gender == 2: # Female
        return 0
    else:
        pass

In [181]:
def Dummy_License(License):

    if License == 1: # Own
        return 1
    elif License == 2: # Do not own
        return 0
    else:
        pass

In [182]:
def Dummy_School_1(School):
    if School == 1: #p Preschooler
        return 1
    else:
        return 0

def Dummy_School_2(School):
    if School == 2: # Elementary School Student
        return 1
    else:
        return 0

def Dummy_School_3(School):
    if School == 3: # Middle(high) School Student
        return 1
    else:
        return 0

def Dummy_School_4(School):
    if School == 4: # Undergraduate(Graduate)
        return 1
    else:
        return 0

def Dummy_School_5(School):
    if School == 5: # Not a Student
        return 1
    else:
        return 0

In [183]:
def Dummy_Job_1(Job):
    if Job == 1: # Specialized Job
        return 1
    else:
        return 0

def Dummy_Job_2(Job):
    if Job == 2: # Service
        return 1
    else:
        return 0

def Dummy_Job_3(Job):
    if Job == 3: # Retail
        return 1
    else:
        return 0

def Dummy_Job_4(Job):
    if Job == 4: # Management or office
        return 1
    else:
        return 0

def Dummy_Job_5(Job):
    if Job == 5: # Agriculture
        return 1
    else:
        return 0

def Dummy_Job_6(Job):
    if Job == 6: # Simple Labor
        return 1
    else:
        return 0

def Dummy_Job_7(Job):
    if Job == 7: # homemaker
        return 1
    else:
        return 0

def Dummy_Job_8(Job):
    if Job == 8: # unemployed or student
        return 1
    else:
        return 0

def Dummy_Job_9(Job):
    if Job == 9: # etc.
        return 1
    else:
        return 0

In [184]:
def Dummy_Workdays_1(Workdays):
    if Workdays == 1: # 6~7 day weekly
        return 1
    else:
        return 0

def Dummy_Workdays_2(Workdays):
    if Workdays == 2: # 5 day weekly
        return 1
    else:
        return 0

def Dummy_Workdays_3(Workdays):
    if Workdays == 3: # 3~4 days weekly
        return 1
    else:
        return 0

def Dummy_Workdays_4(Workdays):
    if Workdays == 4: # 1~2 days weekly
        return 1
    else:
        return 0

def Dummy_Workdays_5(Workdays):
    if Workdays == 5: # etc.
        return 1
    else:
        return 0

In [185]:
def Dummy_Workhours_1(Workhours):
    if Workhours == 1: # Full time
        return 1
    else:
        return 0

def Dummy_Workhours_2(Workhours):
    if Workhours == 2: # part-time(less than 8 hours)
        return 1
    else:
        return 0

def Dummy_Workhours_3(Workhours):
    if Workhours == 3: # etc(including irregular work)
        return 1
    else:
        return 0

In [186]:
def Dummy_Trip_made(Trip_made):
    if Trip_made == 1: # Traveled
        return 1
    elif Trip_made == 2: # Not Traveled
        return 0
    else:
        pass

In [187]:
def Dummy_Trip_d_P_1(Trip_d_P):
    if Trip_d_P == 1: # AM
        return 1
    else:
        return 0
    
def Dummy_Trip_d_P_2(Trip_d_P):
    if Trip_d_P == 2: # PM
        return 1
    else:
        return 0
    
def Dummy_Trip_d_P_3(Trip_d_P):
    if Trip_d_P == 3: # AM(following day)
        return 1
    else:
        return 0

In [188]:
df['Car_ownership'] = df['Ncars'].apply(Dummy_car_ownership)

df['HHrel_1'] = df['HHrel'].apply(Dummy_HHrel_1)
df['HHrel_2'] = df['HHrel'].apply(Dummy_HHrel_2)
df['HHrel_3'] = df['HHrel'].apply(Dummy_HHrel_3)
df['HHrel_4'] = df['HHrel'].apply(Dummy_HHrel_4)
df['HHrel_5'] = df['HHrel'].apply(Dummy_HHrel_5)

df['Gender_DM'] = df['Gender'].apply(Dummy_Gender)
df['License_DM'] = df['Licence'].apply(Dummy_License)

df['School_1'] = df['School'].apply(Dummy_School_1)
df['School_2'] = df['School'].apply(Dummy_School_2)
df['School_3'] = df['School'].apply(Dummy_School_3)
df['School_4'] = df['School'].apply(Dummy_School_4)
df['School_5'] = df['School'].apply(Dummy_School_5)

df['Job_1'] = df['Job'].apply(Dummy_Job_1)
df['Job_2'] = df['Job'].apply(Dummy_Job_2)
df['Job_3'] = df['Job'].apply(Dummy_Job_3)
df['Job_4'] = df['Job'].apply(Dummy_Job_4)
df['Job_5'] = df['Job'].apply(Dummy_Job_5)
df['Job_6'] = df['Job'].apply(Dummy_Job_6)
df['Job_7'] = df['Job'].apply(Dummy_Job_7)
df['Job_8'] = df['Job'].apply(Dummy_Job_8)
df['Job_9'] = df['Job'].apply(Dummy_Job_9)

df['Workdays_1'] = df['Workdays'].apply(Dummy_Workdays_1)
df['Workdays_2'] = df['Workdays'].apply(Dummy_Workdays_2)
df['Workdays_3'] = df['Workdays'].apply(Dummy_Workdays_3)
df['Workdays_4'] = df['Workdays'].apply(Dummy_Workdays_4)
df['Workdays_5'] = df['Workdays'].apply(Dummy_Workdays_5)

df['Workhours_1'] = df['Workhours'].apply(Dummy_Workhours_1)
df['Workhours_2'] = df['Workhours'].apply(Dummy_Workhours_2)
df['Workhours_3'] = df['Workhours'].apply(Dummy_Workhours_3)

df['Trip_d_P_1'] = df['Trip_d_P'].apply(Dummy_Trip_d_P_1)
df['Trip_d_P_2'] = df['Trip_d_P'].apply(Dummy_Trip_d_P_2)
df['Trip_d_P_3'] = df['Trip_d_P'].apply(Dummy_Trip_d_P_3)

In [189]:
df.head(3)

,No,HH_ID,HHsize,HHsize5,HHinc,Ncars,P_ID,HHrel,Age,Gender,...,Workdays_2,Workdays_3,Workdays_4,Workdays_5,Workhours_1,Workhours_2,Workhours_3,Trip_d_P_1,Trip_d_P_2,Trip_d_P_3
0,1,4889,2,2,4,1,1,1,36,1,...,1,0,0,0,1,0,0,1,0,0
1,2,4889,2,2,4,1,1,1,36,1,...,1,0,0,0,1,0,0,0,1,0
2,3,4889,2,2,4,1,2,2,36,2,...,1,0,0,0,1,0,0,1,0,0


### Create Trip Time Variable

In [190]:
def Trip_time_calculate(Trip_a_P, Trip_a_hh, Trip_a_mm, Trip_d_P, Trip_d_hh, Trip_d_mm):
    Trip_d = Trip_d_hh * 60 + Trip_d_mm
    Trip_a = Trip_a_hh * 60 + Trip_a_mm
    
    if Trip_a_P == Trip_d_P: # 둘의 오전, 오후 포함 시간대가 같은 경우
        Trip_time = Trip_a - Trip_d
        
    elif Trip_a_P == 1 and Trip_d_P == 2: #오전 출발, 오후 도착
        Trip_time = (Trip_a + 12 * 60) - Trip_d
        
    elif Trip_a_P == 2 and Trip_d_P == 3: #오후 출발, 다음날 도착
        Trip_time = (Trip_a + 24 * 60) - (Trip_d + 12 * 60)
        
    elif Trip_a_P == 1 and Trip_d_P == 3: #오전 출발, 다음날 도착
        Trip_time = (Trip_a + 24 * 60) - Trip_d
        
    else:
        Trip_time = None
        
    return Trip_time

In [191]:
df['Trip_time'] = df.apply(lambda x : Trip_time_calculate(x['Trip_a_P'], x['Trip_a_hh'], x['Trip_a_mm'], x['Trip_d_P'], x['Trip_d_hh'], x['Trip_d_mm']) , axis = 1 )

In [192]:
df.head(3)

,No,HH_ID,HHsize,HHsize5,HHinc,Ncars,P_ID,HHrel,Age,Gender,...,Workdays_3,Workdays_4,Workdays_5,Workhours_1,Workhours_2,Workhours_3,Trip_d_P_1,Trip_d_P_2,Trip_d_P_3,Trip_time
0,1,4889,2,2,4,1,1,1,36,1,...,0,0,0,1,0,0,1,0,0,60.0
1,2,4889,2,2,4,1,1,1,36,1,...,0,0,0,1,0,0,0,1,0,40.0
2,3,4889,2,2,4,1,2,2,36,2,...,0,0,0,1,0,0,1,0,0,30.0


### Create Trip Purpose

In [193]:
def Trip_purpose(Trip_purpG):
    if Trip_purpG == 1:
        return 'Work'
    
    elif Trip_purpG == 2:
        return 'Education'
    
    elif Trip_purpG == 3:
        return 'Business'
    
    elif Trip_purpG == 4:
        return 'Shopping/social/leisure'
    
    elif Trip_purpG == 5:
        return 'Back-home'
    
    elif Trip_purpG == 6:
        return 'Other'
    
    else:
        pass

In [194]:
df['Trip_Purpose'] = df['Trip_purpG'].apply(Trip_purpose)

### Data Filtering

In [195]:
full = df[df['Trip_made'] == 1]

In [196]:
full.dropna(inplace = True)

<ipython-input-196-53a8ed2d1e32>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  full.dropna(inplace = True)


## Decision Tree
* forecasting **trip purpose**
* Variables:
    * Household Information : 
    * Household Member's Information :
    * Trip Information : 
* Apply F1-Score as the measurement of the performance

### Partitioning dataset into training set / test set

In [215]:
feature_names = [
    #'HHsize', 'HHinc',
    'Age',
    #'Trip_time',
    'Car_ownership',
    'HHrel_1', #'HHrel_2', 'HHrel_3', 'HHrel_4', 'HHrel_5',
    'Gender_DM',
    #'License_DM',
    'School_5',
    'Job_2', 'Job_3', 'Job_4', 'Job_5', 'Job_6', 'Job_7', 'Job_8', 'Job_9',
    #'Workdays_1', #'Workdays_3', 'Workdays_4', 'Workdays_5',
    'Workhours_1', #'Workhours_3',
    #'Trip_d_P_1', 
    'Trip_d_P_2', 'Trip_d_P_3'
]

In [216]:
X = full[feature_names]

In [217]:
y = full['Trip_Purpose']

In [218]:
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                   test_size = 0.3,
                                                   random_state = 42,
                                                   stratify = y)

### Constructing Tree

In [219]:
DT = tree.DecisionTreeClassifier(criterion = 'gini', max_depth = 32, random_state = 42)

### Learning

In [220]:
DT.fit(X_train, y_train)

DecisionTreeClassifier(max_depth=32, random_state=42)

### Plotting Tree

dot_data = export_graphviz(DT, out_file = None, 
                      feature_names = feature_names,  
                      class_names = ['Work', 'Education', 'Business', 'Shopping/social/leisure', 'Back-home', 'Other'],  
                      filled = True, rounded = True,  
                      special_characters = True)

graph = graph_from_dot_data(dot_data)  
graph.write_png('DT.png', scale = 0.15)

### Evaluation

In [221]:
y_pred = DT.predict(X_test)

In [222]:
y_pred

array(['Back-home', 'Back-home', 'Back-home', ..., 'Work', 'Back-home',
       'Work'], dtype=object)

### Confusion Matrix & Measurements

In [223]:
# Confusion matrix & measurements
print(confusion_matrix(y_test, y_pred))
print('')
print('=' * 70)
print('')
print(classification_report(y_test, y_pred))

[[2705   17    0    9   25   36]
 [ 178    5    0    0    1   41]
 [   2    0    4    0    0    1]
 [  73    1    0    1    1   32]
 [ 239    1    0    0    0   16]
 [ 102   10    1   13    2 2574]]


                         precision    recall  f1-score   support

              Back-home       0.82      0.97      0.89      2792
               Business       0.15      0.02      0.04       225
              Education       0.80      0.57      0.67         7
                  Other       0.04      0.01      0.02       108
Shopping/social/leisure       0.00      0.00      0.00       256
                   Work       0.95      0.95      0.95      2702

               accuracy                           0.87      6090
              macro avg       0.46      0.42      0.43      6090
           weighted avg       0.81      0.87      0.83      6090

